In [1]:
import pandas as pd
import glob
import sys
import os
pd.set_option('display.max_rows', 500)

# classification_task_result_files = '/home/ymyung/projects/deeppk/3_Results/classification_MPNN_results/1_Basic'
# regression_task_result_files = '/home/ymyung/projects/deeppk/3_Results/regression_MPNN_results/1_Basic'

# interpretable_results_class = '/home/ymyung/projects/deeppk/interpretable_admet_performance_class.csv'
# interpretable_results_reg = '/home/ymyung/projects/deeppk/interpretable_admet_performance_reg.csv'
all_results_class = '/home/ymyung/projects/deeppk/all_performance_class.csv'
all_results_reg = '/home/ymyung/projects/deeppk/all_performance_reg.csv'

# class_file_list = glob.glob(os.path.join(classification_task_result_files,'**/**/**_scores.json'))
# reg_file_list = glob.glob(os.path.join(regression_task_result_files,'**/**/**_scores.json'))


# dir_path = '/home/ymyung/projects/deeppk/3_Results/All'
# dir_path = '/home/ymyung/projects/deeppk/3_Results/All_Random_300EPOCH'
dir_path = '/home/ymyung/projects/deeppk/3_Results/all_new'
all_results_list= glob.glob(os.path.join(dir_path,'**','**_scores.json'), recursive=True)

reg_file_list = list()
class_file_list = list()

for each_csv in all_results_list:
    if 'accuracy' in pd.read_json(each_csv).columns:
        class_file_list.append(each_csv)
    else:
        reg_file_list.append(each_csv)


### Score Retriving Methods for in-house Classification/Regression results

In [14]:
def get_class_scores(class_file_list, type_of_run):
    class_summary_pd = pd.DataFrame()
    class_file_list = [c_file for c_file in class_file_list if type_of_run.lower() in c_file]

    for _json in class_file_list:
        print(_json)
        if 'fold_4' in _json:
            print(_json)
            # _property_name = _json.split('/fold_0')[0].split('/All/')[1]
            _property_name = _json.split('/fold_4')[0].split('/all_new/')[1]


            score_pd = pd.read_json(_json)
            score_pd['property'] = _property_name
            score_pd['database'] = 'MPNN-{}'.format(type_of_run)
            score_pd.set_index('property',inplace=True)
            class_summary_pd = pd.concat([class_summary_pd,score_pd],axis=0)

        return class_summary_pd

def get_reg_scores(reg_file_list, type_of_run):
    reg_summary_pd = pd.DataFrame()
    reg_file_list = [c_file for c_file in reg_file_list if type_of_run.lower() in c_file]

    for _json in reg_file_list:
        if 'fold_4' in _json:
            # _property_name = _json.split('/fold_0')[0].split('/All/')[1]
            _property_name = _json.split('/fold_4')[0].split('/all_new/')[1]

            score_pd = pd.read_json(_json)
            score_pd['property'] = _property_name
            score_pd['database'] = 'MPNN-{}'.format(type_of_run)
            score_pd.set_index('property',inplace=True)
            reg_summary_pd = pd.concat([reg_summary_pd,score_pd],axis=0)

        return reg_summary_pd

### Score Retriving Methods for Interpretable-ADMET Classification/Regression results

In [7]:
def get_class_scores_interpretable(result_csv):
    result_pd = pd.read_csv(result_csv)
    return result_pd

### Methods for comparing in-house and Interpretable-ADMET results

In [8]:
def compare_class(class_file_list,interpretable_results_class,type_of_run):
    out_pd = pd.DataFrame()

    our_pd = get_class_scores(class_file_list,type_of_run)
    int_pd = get_class_scores_interpretable(interpretable_results_class)
    int_pd = int_pd.query('Database.str.contains(@type_of_run)')

    for group_id, group_cont in int_pd.groupby(by='Database'):
        # print(our_pd.query('index == "caco_2"'))
        # print(group_cont.set_index('property').index)

        _pd = pd.concat([our_pd,group_cont.set_index('Property')], axis=1)
        _pd['d_ACC'] = round(_pd['accuracy'] - _pd['ACC'],3)
        _pd['d_MCC'] = round(_pd['mcc'] - _pd['MCC'],3)
        _pd['d_AUC'] = round(_pd['auc'] - _pd['AUC'],3)

        out_pd = pd.concat([out_pd,_pd])

    out_pd.drop_duplicates(inplace=True)
    out_pd.dropna(subset=['mcc'],inplace=True)
    return out_pd

def compare_reg(reg_file_list,interpretable_results_reg,type_of_run):
    out_pd = pd.DataFrame()

    our_pd = get_reg_scores(reg_file_list,type_of_run)
    int_pd = get_class_scores_interpretable(interpretable_results_reg)

    int_pd = int_pd.query('Database.str.contains(@type_of_run)')

    for group_id, group_cont in int_pd.groupby(by='Database'):
    # for group_id, group_cont in int_pd.groupby(by='Database'):

        _pd = pd.concat([our_pd,group_cont.set_index('Property')],axis=1)
        # _pd = _pd.dropna()
        _pd['d_RMSE'] = round(_pd['rmse'] - _pd['RMSE'],3)
        _pd['d_R2'] = round(_pd['r2'] - _pd['Q2'],3)
        # print(_pd)
        out_pd = pd.concat([out_pd,_pd])

    out_pd.drop_duplicates(inplace=True)
    return out_pd


In [15]:
compare_class(class_file_list,all_results_class,'Test').sort_values(by='d_MCC', ascending=False).sort_index().dropna(subset=['accuracy'])
# compare_reg(reg_file_list,all_results_reg, 'Test').sort_values(by='d_R2', ascending=False).sort_index().dropna(subset=['r2'])

/home/ymyung/projects/deeppk/3_Results/all_new/fdamdd_class/fold_3/test_scores.json


KeyError: 'accuracy'